In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import gseapy as gp
import matplotlib.pyplot as plt
import swan_vis as swan
import yaml
from snakemake.io import expand

p = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.append(p)

from proc_revisions.utils import *
from proc_revisions.plotting import *

In [11]:
od = '../../proc_revisions/'
config_file = f'{od}/config.yml'
with open(config_file) as f:
    config = yaml.safe_load(f)

In [12]:
ab = od+expand(config['lr']['talon']['fusion_fix']['ab'], species='human')[0]
unfilt_ab = od+expand(config['lr']['cerberus']['ab'], species='human')[0] 
filt_ab = od+expand(config['lr']['cerberus']['filt_ab'], species='human')[0]
read_annot = od+expand(config['lr']['talon']['full_annot'], species='human')[0]
t_metadata = od+expand(config['ref']['cerberus']['new_gtf_t_info'], species='human')[0]
lib_meta = od+expand(config['lr']['meta'], species='human')[0]
swan_file = od+expand(config['lr']['swan']['sg'], species='human')[0]
cerberus_h5 = od+expand(config['lr']['cerberus']['ca_triplets'], species='human', obs_col='sample')[0]
cerb_t_metadata = od+expand(config['lr']['cerberus']['gtf_t_info'], species='human')[0]
major_isos = od+expand(config['lr']['analysis']['major_isos'], species='human', obs_col='sample')[0]
pi_tpm_table = od+expand(config['lr']['mane']['pi_tpm']['triplet'], species='human', obs_col='sample')[0]
gtex_gtf = od+expand(config['gtex']['filt_gtf'], species='human')[0]
gtex_ab = od+expand(config['gtex']['ab_fmt'], species='human')[0]
gtex_cerb_ab = od+expand(config['gtex']['cerberus']['ab'], species='human')[0]

ref_t_metadata = od+expand(config['ref']['new_gtf_t_info'], species='human')[0]
ref_g_metadata = od+expand(config['ref']['new_gtf_g_info'], species='human')[0]

ver = 'v40_cerberus'
min_tpm = 1
gene_subset = 'polya'
obs_col = 'sample'
go_gene_subset = 'protein_coding'
predom_iso_subset = 'protein_coding'

m_lib_meta = od+expand(config['lr']['meta'], species='mouse')[0]

In [4]:
ca = cerberus.read(cerberus_h5)

In [6]:
ca.t_map.source.unique()

array(['v40', 'v29', 'gtex', 'lapa'], dtype=object)

In [9]:
cerberus.replace_gtf_ids(cerberus_h5,
                         gtex_gtf,
                        'gtex',
                         True,
                         True,
                         'test_gtex_replace_gtf_ids.gtf')

1704222921.489065:INFO:Cerberus:cerberus:Adding Cerberus transcript ids


> /Users/fairliereese/Documents/programming/mortazavi_lab/bin/cerberus/cerberus/cerberus.py(2541)replace_gtf_ids()
   2539     import pdb; pdb.set_trace()
   2540 
-> 2541     df.drop(['transcript_id', 'transcript_name'], axis=1, inplace=True)
   2542     df.rename({'transcript_id_cerberus': 'transcript_id',
   2543                'transcript_name_cerberus': 'transcript_name'},



ipdb>  c


1704222928.757854:INFO:Cerberus:cerberus:Updating ends of transcripts...
1704222943.801610:INFO:Cerberus:cerberus:Deduplicating transcripts...


## something going on w/ the abundance aggregation too

Maybe the transcript IDs were not the same format between the GTF and the abundance matrix?

In [42]:
df = pd.read_csv(gtex_ab, sep='\t')
ca = cerberus.read(cerberus_h5)
ca_df = ca.t_map.loc[ca.t_map.source=='gtex']

In [47]:
ca_df.head()

,original_transcript_id,ic,ic_id,tss_id,tss,tes_id,tes,gene_id,gene_name,original_transcript_name,transcript_triplet,transcript_id,transcript_name,tss_first_sd_issue,tes_last_sa_issue,source
0,0016cd97-0137-4194-9821-910931c6e972_ENSG00000...,24,ENSG00000086015_24,ENSG00000086015_5,5.0,ENSG00000086015_1,1.0,ENSG00000086015,MAST2,0016cd97-0137-4194-9821-910931c6e972_ENSG00000...,"[5,24,1]","ENSG00000086015[5,24,1]","MAST2[5,24,1]",False,False,gtex
1,001eea3f-368c-444f-96c0-539cb2204403_ENSG00000...,28,ENSG00000162736_28,ENSG00000162736_1,1.0,ENSG00000162736_1,1.0,ENSG00000162736,NCSTN,001eea3f-368c-444f-96c0-539cb2204403_ENSG00000...,"[1,28,1]","ENSG00000162736[1,28,1]","NCSTN[1,28,1]",False,False,gtex
2,0039fbf5-7334-40e1-a1a9-2aa34b6e009c_ENSG00000...,23,ENSG00000084072_23,ENSG00000084072_1,1.0,ENSG00000084072_3,3.0,ENSG00000084072,PPIE,0039fbf5-7334-40e1-a1a9-2aa34b6e009c_ENSG00000...,"[1,23,3]","ENSG00000084072[1,23,3]","PPIE[1,23,3]",False,False,gtex
3,003cbbd1-0cb2-43d0-ba48-daabd4d7d518_ENSG00000...,11,ENSG00000137965_11,ENSG00000137965_1,1.0,ENSG00000137965_1,1.0,ENSG00000137965,IFI44,003cbbd1-0cb2-43d0-ba48-daabd4d7d518_ENSG00000...,"[1,11,1]","ENSG00000137965[1,11,1]","IFI44[1,11,1]",False,False,gtex
4,004a6304-0523-477e-ad32-90215c977dca_ENSG00000...,77,ENSG00000127603_77,ENSG00000127603_39,39.0,ENSG00000127603_36,36.0,ENSG00000127603,MACF1,004a6304-0523-477e-ad32-90215c977dca_ENSG00000...,"[39,77,36]","ENSG00000127603[39,77,36]","MACF1[39,77,36]",False,False,gtex


In [44]:
df['tid'] = df.transcript_ID.str.rsplit('_', n=1, expand=True)[0]
df.head()

,GTEX-1192X-0011-R10a-SM-4RXXZ,GTEX-11H98-0011-R11b-SM-4SFLZ,GTEX-11TTK-0011-R7b-SM-4TVFS,GTEX-1211K-0826-SM-7LDFQ,GTEX-1313W-0011-R7b-SM-4ZL3U,GTEX-13QBU-0426-SM-5A4VT,GTEX-13QJ3-0726-SM-7LDHS,GTEX-13QJ3-0726-SM-7LDHS_rep,GTEX-13RTJ-0011-R7b-SM-5CTCB,GTEX-13VXU-0011-R11b-SM-5BFQZ,...,GTEX-S4Z8-0008-SM-2Y983_exp2,GTEX-S95S-0008-SM-3RQ8B_ctrl,GTEX-S95S-0008-SM-3RQ8B_exp1,GTEX-S95S-0008-SM-3RQ8B_exp2,GTEX-WY7C-0008-SM-3NZB5_ctrl,GTEX-WY7C-0008-SM-3NZB5_exp,annot_transcript_id,annot_transcript_name,transcript_ID,tid
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.271232,...,0.000000,0.000000,0.000000,0.262969,0.000000,0.397173,000187c4-a488-40f0-a69c-0a89582f3241_ENSG00000...,000187c4-a488-40f0-a69c-0a89582f3241_ENSG00000...,000187c4-a488-40f0-a69c-0a89582f3241_ENSG00000...,000187c4-a488-40f0-a69c-0a89582f3241
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.278141,0.294667,0.000000,0.262969,0.000000,0.397173,00026598-3078-4e2f-8ac9-dd8f523396b9_ENSG00000...,00026598-3078-4e2f-8ac9-dd8f523396b9_ENSG00000...,00026598-3078-4e2f-8ac9-dd8f523396b9_ENSG00000...,00026598-3078-4e2f-8ac9-dd8f523396b9_ENSG00000...
2,0.380702,5.359084,1.961873,4.590649,1.793796,7.308494,6.346269,2.378808,0.653233,1.356161,...,10.013078,14.144038,13.873059,13.937337,1.727199,11.518015,0002a5e2-f01a-4690-a7db-7af726712a5e_ENSG00000...,0002a5e2-f01a-4690-a7db-7af726712a5e_ENSG00000...,0002a5e2-f01a-4690-a7db-7af726712a5e_ENSG00000...,0002a5e2-f01a-4690-a7db-7af726712a5e_ENSG00000...
3,0.000000,0.297727,0.000000,1.020144,0.000000,0.000000,0.000000,0.000000,1.959698,0.542464,...,1.112564,0.294667,0.867066,0.525937,0.431800,1.191519,000339f1-1769-4608-b369-59aa222cd7b7_ENSG00000...,000339f1-1769-4608-b369-59aa222cd7b7_ENSG00000...,000339f1-1769-4608-b369-59aa222cd7b7_ENSG00000...,000339f1-1769-4608-b369-59aa222cd7b7_ENSG00000...
4,14.847375,18.459067,17.656857,4.080577,10.762776,13.288171,15.577205,11.418279,11.104956,20.613647,...,6.397244,7.366687,6.936529,9.203902,9.067796,7.149113,0003706a-94a7-4419-a61d-6310d7a9c10c_ENSG00000...,0003706a-94a7-4419-a61d-6310d7a9c10c_ENSG00000...,0003706a-94a7-4419-a61d-6310d7a9c10c_ENSG00000...,0003706a-94a7-4419-a61d-6310d7a9c10c_ENSG00000...


In [50]:
# how many of these "tids" intersect with the ones in the GTF?
print(len(df.index))
print(len(df.tid.unique().tolist()))
print(len(ca_df.index))
print(len(ca_df.original_transcript_id.unique().tolist()))

print(len(set(df.tid.tolist())&set(ca_df.original_transcript_id.tolist())))
# this is fine IMO 

93630
93630
88312
88312
88252


In [46]:
# # which entryes have - and _?
# df.loc[(df.transcript_ID.str.contains('_'))&(df.transcript_ID.str.contains('-'))]
# df.head()

In [15]:
df = pd.read_csv(gtex_ab, sep='\t')
ca = cerberus.read(cerberus_h5)
df2 = ca.t_map.loc[ca.t_map.source=='gtex']

In [16]:
df.head()

,GTEX-1192X-0011-R10a-SM-4RXXZ,GTEX-11H98-0011-R11b-SM-4SFLZ,GTEX-11TTK-0011-R7b-SM-4TVFS,GTEX-1211K-0826-SM-7LDFQ,GTEX-1313W-0011-R7b-SM-4ZL3U,GTEX-13QBU-0426-SM-5A4VT,GTEX-13QJ3-0726-SM-7LDHS,GTEX-13QJ3-0726-SM-7LDHS_rep,GTEX-13RTJ-0011-R7b-SM-5CTCB,GTEX-13VXU-0011-R11b-SM-5BFQZ,...,GTEX-S4Z8-0008-SM-2Y983_exp1,GTEX-S4Z8-0008-SM-2Y983_exp2,GTEX-S95S-0008-SM-3RQ8B_ctrl,GTEX-S95S-0008-SM-3RQ8B_exp1,GTEX-S95S-0008-SM-3RQ8B_exp2,GTEX-WY7C-0008-SM-3NZB5_ctrl,GTEX-WY7C-0008-SM-3NZB5_exp,annot_transcript_id,annot_transcript_name,transcript_ID
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.271232,...,0.971980,0.000000,0.000000,0.000000,0.262969,0.000000,0.397173,000187c4-a488-40f0-a69c-0a89582f3241_ENSG00000...,000187c4-a488-40f0-a69c-0a89582f3241_ENSG00000...,000187c4-a488-40f0-a69c-0a89582f3241_ENSG00000...
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.278141,0.294667,0.000000,0.262969,0.000000,0.397173,00026598-3078-4e2f-8ac9-dd8f523396b9_ENSG00000...,00026598-3078-4e2f-8ac9-dd8f523396b9_ENSG00000...,00026598-3078-4e2f-8ac9-dd8f523396b9_ENSG00000...
2,0.380702,5.359084,1.961873,4.590649,1.793796,7.308494,6.346269,2.378808,0.653233,1.356161,...,12.635743,10.013078,14.144038,13.873059,13.937337,1.727199,11.518015,0002a5e2-f01a-4690-a7db-7af726712a5e_ENSG00000...,0002a5e2-f01a-4690-a7db-7af726712a5e_ENSG00000...,0002a5e2-f01a-4690-a7db-7af726712a5e_ENSG00000...
3,0.000000,0.297727,0.000000,1.020144,0.000000,0.000000,0.000000,0.000000,1.959698,0.542464,...,0.971980,1.112564,0.294667,0.867066,0.525937,0.431800,1.191519,000339f1-1769-4608-b369-59aa222cd7b7_ENSG00000...,000339f1-1769-4608-b369-59aa222cd7b7_ENSG00000...,000339f1-1769-4608-b369-59aa222cd7b7_ENSG00000...
4,14.847375,18.459067,17.656857,4.080577,10.762776,13.288171,15.577205,11.418279,11.104956,20.613647,...,10.691783,6.397244,7.366687,6.936529,9.203902,9.067796,7.149113,0003706a-94a7-4419-a61d-6310d7a9c10c_ENSG00000...,0003706a-94a7-4419-a61d-6310d7a9c10c_ENSG00000...,0003706a-94a7-4419-a61d-6310d7a9c10c_ENSG00000...


In [17]:
df2.head()

,original_transcript_id,ic,ic_id,tss_id,tss,tes_id,tes,gene_id,gene_name,original_transcript_name,transcript_triplet,transcript_id,transcript_name,tss_first_sd_issue,tes_last_sa_issue,source
0,0016cd97-0137-4194-9821-910931c6e972_ENSG00000...,24,ENSG00000086015_24,ENSG00000086015_5,5.0,ENSG00000086015_1,1.0,ENSG00000086015,MAST2,0016cd97-0137-4194-9821-910931c6e972_ENSG00000...,"[5,24,1]","ENSG00000086015[5,24,1]","MAST2[5,24,1]",False,False,gtex
1,001eea3f-368c-444f-96c0-539cb2204403_ENSG00000...,28,ENSG00000162736_28,ENSG00000162736_1,1.0,ENSG00000162736_1,1.0,ENSG00000162736,NCSTN,001eea3f-368c-444f-96c0-539cb2204403_ENSG00000...,"[1,28,1]","ENSG00000162736[1,28,1]","NCSTN[1,28,1]",False,False,gtex
2,0039fbf5-7334-40e1-a1a9-2aa34b6e009c_ENSG00000...,23,ENSG00000084072_23,ENSG00000084072_1,1.0,ENSG00000084072_3,3.0,ENSG00000084072,PPIE,0039fbf5-7334-40e1-a1a9-2aa34b6e009c_ENSG00000...,"[1,23,3]","ENSG00000084072[1,23,3]","PPIE[1,23,3]",False,False,gtex
3,003cbbd1-0cb2-43d0-ba48-daabd4d7d518_ENSG00000...,11,ENSG00000137965_11,ENSG00000137965_1,1.0,ENSG00000137965_1,1.0,ENSG00000137965,IFI44,003cbbd1-0cb2-43d0-ba48-daabd4d7d518_ENSG00000...,"[1,11,1]","ENSG00000137965[1,11,1]","IFI44[1,11,1]",False,False,gtex
4,004a6304-0523-477e-ad32-90215c977dca_ENSG00000...,77,ENSG00000127603_77,ENSG00000127603_39,39.0,ENSG00000127603_36,36.0,ENSG00000127603,MACF1,004a6304-0523-477e-ad32-90215c977dca_ENSG00000...,"[39,77,36]","ENSG00000127603[39,77,36]","MACF1[39,77,36]",False,False,gtex


In [18]:
len(set(df2.original_transcript_id.tolist())-set(df.transcript_ID.tolist()))

88311

In [19]:
df.loc[df.transcript_ID.str.contains('0016cd97-0137-4194-9821-910931c6e972')]

,GTEX-1192X-0011-R10a-SM-4RXXZ,GTEX-11H98-0011-R11b-SM-4SFLZ,GTEX-11TTK-0011-R7b-SM-4TVFS,GTEX-1211K-0826-SM-7LDFQ,GTEX-1313W-0011-R7b-SM-4ZL3U,GTEX-13QBU-0426-SM-5A4VT,GTEX-13QJ3-0726-SM-7LDHS,GTEX-13QJ3-0726-SM-7LDHS_rep,GTEX-13RTJ-0011-R7b-SM-5CTCB,GTEX-13VXU-0011-R11b-SM-5BFQZ,...,GTEX-S4Z8-0008-SM-2Y983_exp1,GTEX-S4Z8-0008-SM-2Y983_exp2,GTEX-S95S-0008-SM-3RQ8B_ctrl,GTEX-S95S-0008-SM-3RQ8B_exp1,GTEX-S95S-0008-SM-3RQ8B_exp2,GTEX-WY7C-0008-SM-3NZB5_ctrl,GTEX-WY7C-0008-SM-3NZB5_exp,annot_transcript_id,annot_transcript_name,transcript_ID
20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.475762,0.653233,0.0,...,0.0,0.0,0.294667,0.433533,0.262969,0.0,0.0,0016cd97-0137-4194-9821-910931c6e972_ENSG00000...,0016cd97-0137-4194-9821-910931c6e972_ENSG00000...,0016cd97-0137-4194-9821-910931c6e972_ENSG00000...


In [20]:
df2.original_transcript_id.tolist()[0]

'0016cd97-0137-4194-9821-910931c6e972_ENSG00000086015.20'

In [21]:
df.loc[df.transcript_ID=='0016cd97-0137-4194-9821-910931c6e972_ENSG00000086015.20']

,GTEX-1192X-0011-R10a-SM-4RXXZ,GTEX-11H98-0011-R11b-SM-4SFLZ,GTEX-11TTK-0011-R7b-SM-4TVFS,GTEX-1211K-0826-SM-7LDFQ,GTEX-1313W-0011-R7b-SM-4ZL3U,GTEX-13QBU-0426-SM-5A4VT,GTEX-13QJ3-0726-SM-7LDHS,GTEX-13QJ3-0726-SM-7LDHS_rep,GTEX-13RTJ-0011-R7b-SM-5CTCB,GTEX-13VXU-0011-R11b-SM-5BFQZ,...,GTEX-S4Z8-0008-SM-2Y983_exp1,GTEX-S4Z8-0008-SM-2Y983_exp2,GTEX-S95S-0008-SM-3RQ8B_ctrl,GTEX-S95S-0008-SM-3RQ8B_exp1,GTEX-S95S-0008-SM-3RQ8B_exp2,GTEX-WY7C-0008-SM-3NZB5_ctrl,GTEX-WY7C-0008-SM-3NZB5_exp,annot_transcript_id,annot_transcript_name,transcript_ID


In [24]:
df.loc[df.transcript_ID.str.contains('0016cd97-0137-4194-9821-910931c6e972'), 'annot_transcript_name'].tolist()

['0016cd97-0137-4194-9821-910931c6e972_ENSG00000086015.20_ENSG00000086015.20']

In [26]:
df['tid'] = df.transcript_ID.str.rsplit('_', n=1, expand=True)[1]
df.head()

,GTEX-1192X-0011-R10a-SM-4RXXZ,GTEX-11H98-0011-R11b-SM-4SFLZ,GTEX-11TTK-0011-R7b-SM-4TVFS,GTEX-1211K-0826-SM-7LDFQ,GTEX-1313W-0011-R7b-SM-4ZL3U,GTEX-13QBU-0426-SM-5A4VT,GTEX-13QJ3-0726-SM-7LDHS,GTEX-13QJ3-0726-SM-7LDHS_rep,GTEX-13RTJ-0011-R7b-SM-5CTCB,GTEX-13VXU-0011-R11b-SM-5BFQZ,...,GTEX-S4Z8-0008-SM-2Y983_exp2,GTEX-S95S-0008-SM-3RQ8B_ctrl,GTEX-S95S-0008-SM-3RQ8B_exp1,GTEX-S95S-0008-SM-3RQ8B_exp2,GTEX-WY7C-0008-SM-3NZB5_ctrl,GTEX-WY7C-0008-SM-3NZB5_exp,annot_transcript_id,annot_transcript_name,transcript_ID,tid
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.271232,...,0.000000,0.000000,0.000000,0.262969,0.000000,0.397173,000187c4-a488-40f0-a69c-0a89582f3241_ENSG00000...,000187c4-a488-40f0-a69c-0a89582f3241_ENSG00000...,000187c4-a488-40f0-a69c-0a89582f3241_ENSG00000...,ENSG00000173848.18
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.278141,0.294667,0.000000,0.262969,0.000000,0.397173,00026598-3078-4e2f-8ac9-dd8f523396b9_ENSG00000...,00026598-3078-4e2f-8ac9-dd8f523396b9_ENSG00000...,00026598-3078-4e2f-8ac9-dd8f523396b9_ENSG00000...,ENSG00000102893.15
2,0.380702,5.359084,1.961873,4.590649,1.793796,7.308494,6.346269,2.378808,0.653233,1.356161,...,10.013078,14.144038,13.873059,13.937337,1.727199,11.518015,0002a5e2-f01a-4690-a7db-7af726712a5e_ENSG00000...,0002a5e2-f01a-4690-a7db-7af726712a5e_ENSG00000...,0002a5e2-f01a-4690-a7db-7af726712a5e_ENSG00000...,ENSG00000071575.11
3,0.000000,0.297727,0.000000,1.020144,0.000000,0.000000,0.000000,0.000000,1.959698,0.542464,...,1.112564,0.294667,0.867066,0.525937,0.431800,1.191519,000339f1-1769-4608-b369-59aa222cd7b7_ENSG00000...,000339f1-1769-4608-b369-59aa222cd7b7_ENSG00000...,000339f1-1769-4608-b369-59aa222cd7b7_ENSG00000...,ENSG00000166012.16
4,14.847375,18.459067,17.656857,4.080577,10.762776,13.288171,15.577205,11.418279,11.104956,20.613647,...,6.397244,7.366687,6.936529,9.203902,9.067796,7.149113,0003706a-94a7-4419-a61d-6310d7a9c10c_ENSG00000...,0003706a-94a7-4419-a61d-6310d7a9c10c_ENSG00000...,0003706a-94a7-4419-a61d-6310d7a9c10c_ENSG00000...,ENSG00000101247.17


In [29]:
df.transcript_ID.values[0]

'000187c4-a488-40f0-a69c-0a89582f3241_ENSG00000173848.18'

In [28]:
df.transcript_ID.values[1]

'00026598-3078-4e2f-8ac9-dd8f523396b9_ENSG00000102893.15_ENSG00000102893.15'

In [30]:
# can I just use these read names instead?
ca = cerberus.read(cerberus_h5)
ca_df = ca.t_map.loc[ca.t_map.source=='gtex']
ca_df.head()

,original_transcript_id,ic,ic_id,tss_id,tss,tes_id,tes,gene_id,gene_name,original_transcript_name,transcript_triplet,transcript_id,transcript_name,tss_first_sd_issue,tes_last_sa_issue,source
0,0016cd97-0137-4194-9821-910931c6e972_ENSG00000...,24,ENSG00000086015_24,ENSG00000086015_5,5.0,ENSG00000086015_1,1.0,ENSG00000086015,MAST2,0016cd97-0137-4194-9821-910931c6e972_ENSG00000...,"[5,24,1]","ENSG00000086015[5,24,1]","MAST2[5,24,1]",False,False,gtex
1,001eea3f-368c-444f-96c0-539cb2204403_ENSG00000...,28,ENSG00000162736_28,ENSG00000162736_1,1.0,ENSG00000162736_1,1.0,ENSG00000162736,NCSTN,001eea3f-368c-444f-96c0-539cb2204403_ENSG00000...,"[1,28,1]","ENSG00000162736[1,28,1]","NCSTN[1,28,1]",False,False,gtex
2,0039fbf5-7334-40e1-a1a9-2aa34b6e009c_ENSG00000...,23,ENSG00000084072_23,ENSG00000084072_1,1.0,ENSG00000084072_3,3.0,ENSG00000084072,PPIE,0039fbf5-7334-40e1-a1a9-2aa34b6e009c_ENSG00000...,"[1,23,3]","ENSG00000084072[1,23,3]","PPIE[1,23,3]",False,False,gtex
3,003cbbd1-0cb2-43d0-ba48-daabd4d7d518_ENSG00000...,11,ENSG00000137965_11,ENSG00000137965_1,1.0,ENSG00000137965_1,1.0,ENSG00000137965,IFI44,003cbbd1-0cb2-43d0-ba48-daabd4d7d518_ENSG00000...,"[1,11,1]","ENSG00000137965[1,11,1]","IFI44[1,11,1]",False,False,gtex
4,004a6304-0523-477e-ad32-90215c977dca_ENSG00000...,77,ENSG00000127603_77,ENSG00000127603_39,39.0,ENSG00000127603_36,36.0,ENSG00000127603,MACF1,004a6304-0523-477e-ad32-90215c977dca_ENSG00000...,"[39,77,36]","ENSG00000127603[39,77,36]","MACF1[39,77,36]",False,False,gtex


In [33]:
ca_df['read_name'] = ca_df.original_transcript_id.str.rsplit('_', n=1, expand=True)[0]
print(len(ca_df.index))
print(len(ca_df.original_transcript_id.unique().tolist()))
print(len(ca_df.read_name.unique().tolist()))

88312
88312
88311


In [34]:
ca_df.loc[ca_df.original_transcript_id.duplicated(keep=False)].sort_values(by='original_transcript_id')
ca_df.loc[ca_df.read_name.duplicated(keep=False)].sort_values(by='read_name')

,original_transcript_id,ic,ic_id,tss_id,tss,tes_id,tes,gene_id,gene_name,original_transcript_name,transcript_triplet,transcript_id,transcript_name,tss_first_sd_issue,tes_last_sa_issue,source,read_name
6484,8e2e0686-77f7-45b3-89e0-6dafafb0664a,127,ENSG00000118194_127,ENSG00000118194_1,1.0,ENSG00000118194_1,1.0,ENSG00000118194,TNNT2,8e2e0686-77f7-45b3-89e0-6dafafb0664a,"[1,127,1]","ENSG00000118194[1,127,1]","TNNT2[1,127,1]",False,False,gtex,8e2e0686-77f7-45b3-89e0-6dafafb0664a
6485,8e2e0686-77f7-45b3-89e0-6dafafb0664a_ENSG00000...,137,ENSG00000118194_137,ENSG00000118194_1,1.0,ENSG00000118194_1,1.0,ENSG00000118194,TNNT2,8e2e0686-77f7-45b3-89e0-6dafafb0664a_ENSG00000...,"[1,137,1]","ENSG00000118194[1,137,1]","TNNT2[1,137,1]",False,False,gtex,8e2e0686-77f7-45b3-89e0-6dafafb0664a


In [35]:
ca_df.loc[ca_df.read_name.duplicated(keep=False)].sort_values(by='read_name').original_transcript_id.tolist()

['8e2e0686-77f7-45b3-89e0-6dafafb0664a',
 '8e2e0686-77f7-45b3-89e0-6dafafb0664a_ENSG00000118194.18']

In [36]:
ca_df.loc[~ca_df.original_transcript_id.str.contains('ENSG')]

,original_transcript_id,ic,ic_id,tss_id,tss,tes_id,tes,gene_id,gene_name,original_transcript_name,transcript_triplet,transcript_id,transcript_name,tss_first_sd_issue,tes_last_sa_issue,source,read_name
7,0079335b-c0e2-4340-8567-6e33b2f3f757,30,ENSG00000132849_30,ENSG00000132849_3,3.0,ENSG00000132849_11,11.0,ENSG00000132849,PATJ,0079335b-c0e2-4340-8567-6e33b2f3f757,"[3,30,11]","ENSG00000132849[3,30,11]","PATJ[3,30,11]",False,False,gtex,0079335b-c0e2-4340-8567-6e33b2f3f757
15,013114c8-7785-4602-9dca-603219fd1845,9,ENSG00000198799_9,ENSG00000198799_1,1.0,ENSG00000198799_11,11.0,ENSG00000198799,LRIG2,013114c8-7785-4602-9dca-603219fd1845,"[1,9,11]","ENSG00000198799[1,9,11]","LRIG2[1,9,11]",False,False,gtex,013114c8-7785-4602-9dca-603219fd1845
18,01827248-7a47-4920-9835-d514c9606bcb,45,ENSG00000004487_45,ENSG00000004487_2,2.0,ENSG00000004487_1,1.0,ENSG00000004487,KDM1A,01827248-7a47-4920-9835-d514c9606bcb,"[2,45,1]","ENSG00000004487[2,45,1]","KDM1A[2,45,1]",False,False,gtex,01827248-7a47-4920-9835-d514c9606bcb
19,01a0b4eb-3e35-49bf-bc97-4e1788c9543a,70,ENSG00000160789_70,ENSG00000160789_1,1.0,ENSG00000160789_3,3.0,ENSG00000160789,LMNA,01a0b4eb-3e35-49bf-bc97-4e1788c9543a,"[1,70,3]","ENSG00000160789[1,70,3]","LMNA[1,70,3]",False,False,gtex,01a0b4eb-3e35-49bf-bc97-4e1788c9543a
21,01b7a18b-9f53-4361-bbe3-b396ae0da486,7,ENSG00000162522_7,ENSG00000162522_2,2.0,ENSG00000162522_1,1.0,ENSG00000162522,KIAA1522,01b7a18b-9f53-4361-bbe3-b396ae0da486,"[2,7,1]","ENSG00000162522[2,7,1]","KIAA1522[2,7,1]",False,False,gtex,01b7a18b-9f53-4361-bbe3-b396ae0da486
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88296,f15b3faa-2ad7-47f5-ab52-04ca60af70df,10,ENSG00000114374_10,ENSG00000114374_2,2.0,ENSG00000114374_8,8.0,ENSG00000114374,USP9Y,f15b3faa-2ad7-47f5-ab52-04ca60af70df,"[2,10,8]","ENSG00000114374[2,10,8]","USP9Y[2,10,8]",False,False,gtex,f15b3faa-2ad7-47f5-ab52-04ca60af70df
88299,02eb4919-694b-4bf3-96db-621db54048c6,13,ENSG00000012817_13,ENSG00000012817_2,2.0,ENSG00000012817_2,2.0,ENSG00000012817,KDM5D,02eb4919-694b-4bf3-96db-621db54048c6,"[2,13,2]","ENSG00000012817[2,13,2]","KDM5D[2,13,2]",False,False,gtex,02eb4919-694b-4bf3-96db-621db54048c6
88306,ENST00000329134.9,7,ENSG00000183878_7,ENSG00000183878_1,1.0,ENSG00000183878_5,5.0,ENSG00000183878,UTY,ENST00000329134.9,"[1,7,5]","ENSG00000183878[1,7,5]","UTY[1,7,5]",False,False,gtex,ENST00000329134.9
88307,ENST00000331397.8,6,ENSG00000183878_6,ENSG00000183878_1,1.0,ENSG00000183878_1,1.0,ENSG00000183878,UTY,ENST00000331397.8,"[1,6,1]","ENSG00000183878[1,6,1]","UTY[1,6,1]",False,False,gtex,ENST00000331397.8


In [37]:
ab_df = pd.read_csv(gtex_ab, sep='\t')


In [41]:
ab_df.loc[ab_df.transcript_ID.str.contains('ENSG')]
ab_df.loc[ab_df.transcript_ID.str.startswith('ENSG')]

,GTEX-1192X-0011-R10a-SM-4RXXZ,GTEX-11H98-0011-R11b-SM-4SFLZ,GTEX-11TTK-0011-R7b-SM-4TVFS,GTEX-1211K-0826-SM-7LDFQ,GTEX-1313W-0011-R7b-SM-4ZL3U,GTEX-13QBU-0426-SM-5A4VT,GTEX-13QJ3-0726-SM-7LDHS,GTEX-13QJ3-0726-SM-7LDHS_rep,GTEX-13RTJ-0011-R7b-SM-5CTCB,GTEX-13VXU-0011-R11b-SM-5BFQZ,...,GTEX-S4Z8-0008-SM-2Y983_exp1,GTEX-S4Z8-0008-SM-2Y983_exp2,GTEX-S95S-0008-SM-3RQ8B_ctrl,GTEX-S95S-0008-SM-3RQ8B_exp1,GTEX-S95S-0008-SM-3RQ8B_exp2,GTEX-WY7C-0008-SM-3NZB5_ctrl,GTEX-WY7C-0008-SM-3NZB5_exp,annot_transcript_id,annot_transcript_name,transcript_ID
